In [3]:
# Импорт Pandas для работы с DataFrame
import pandas as pd

In [17]:
# Чтение CSV-файла в DataFrame
df = pd.read_csv("Bank_data.csv")

# Вывод первых 5 строк для проверки
print(df.head())


   RowNumber  CustomerId   Surname  CreditScore Geography  Gender  Age  \
0          1    15634602  Hargrave          619    France  Female   42   
1          2    15647311      Hill          608     Spain  Female   41   
2          3    15619304      Onio          502    France  Female   42   
3          4    15701354      Boni          699    France  Female   39   
4          5    15737888  Mitchell          850     Spain  Female   43   

   Tenure    Balance  NumOfProducts  HasCrCard  IsActiveMember  \
0     2.0       0.00              1          1               1   
1     1.0   83807.86              1          0               1   
2     8.0  159660.80              3          1               0   
3     1.0       0.00              2          0               0   
4     2.0  125510.82              1          1               1   

   EstimatedSalary  Exited  
0        101348.88       1  
1        112542.58       0  
2        113931.57       1  
3         93826.63       0  
4         790

In [59]:
# удаление ненужного столбца
df = df.drop(columns=['RowNumber', 'Surname'])

In [60]:
#Преобразуем все ошибочные типы данных


# Числовые непрерывные признаки (должны быть float)
numeric_float_cols = ['Balance', 'EstimatedSalary']

for col in numeric_float_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')  # Нечисловое → NaN

# Целочисленные признаки (должны быть целыми числами)
numeric_int_cols = ['CreditScore', 'Age', 'Tenure', 'NumOfProducts', 'CustomerId']

for col in numeric_int_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    # Приведём к nullable integer (Int64), чтобы поддерживать NaN
    df[col] = df[col].astype('Int64')

# Преобразование булевого типа в числовой (0 и 1)
df['HasCrCard'] = df['HasCrCard'].astype('Int64')
df['IsActiveMember'] = df['IsActiveMember'].astype('Int64')
df['Exited'] = df['Exited'].astype('Int64')

# Преобразование гендера
df['Gender'] = df['Gender'].map({'Female': 0, 'Male': 1})

# Проверка уникальных значений сран
print(df['Geography'].unique())

# Кодируем страну
df = pd.get_dummies(df, columns=['Geography'], dtype='int64')

# Проверка итоговых типов
print("Итоговые типы данных:")
print(df.dtypes)



['France' 'Spain' 'Germany']
Итоговые типы данных:
CustomerId             Int64
CreditScore            Int64
Gender                 int64
Age                    Int64
Tenure                 Int64
Balance              float64
NumOfProducts          Int64
HasCrCard              Int64
IsActiveMember         Int64
EstimatedSalary      float64
Exited                 Int64
Geography_France       int64
Geography_Germany      int64
Geography_Spain        int64
dtype: object


In [61]:
# Вывод количества пропусков по всем столбцам
print("Пропущенные значения по всем столбцам:")
print(df.isnull().sum())

Пропущенные значения по всем столбцам:
CustomerId             0
CreditScore            0
Gender                 0
Age                    0
Tenure               909
Balance                0
NumOfProducts          0
HasCrCard              0
IsActiveMember         0
EstimatedSalary        0
Exited                 0
Geography_France       0
Geography_Germany      0
Geography_Spain        0
dtype: int64


In [62]:
# Корреляция Пирсона между Tenure и всеми остальными признаками
corr_with_price = df.corr()['Tenure'].sort_values(key=abs, ascending=False)
print(corr_with_price)

Tenure               1.000000
IsActiveMember      -0.032178
HasCrCard            0.027232
CustomerId          -0.021418
Exited              -0.016761
Age                 -0.013134
Gender               0.012634
NumOfProducts        0.011979
EstimatedSalary      0.010520
Balance             -0.007911
Geography_Germany   -0.003299
Geography_France     0.002167
Geography_Spain      0.000810
CreditScore         -0.000062
Name: Tenure, dtype: float64


Tenure не коррелирует с другими параметрами меньше 4%, значит можно заменить на медиану

In [63]:
# Заполним пропуски медианой (лучше всего подходит для Tenure)
df['Tenure'] = df['Tenure'].fillna(df['Tenure'].median()).astype('Int64')

# Проверим, что пропусков больше нет
print("Пропуски после обработки:")
print(df['Tenure'].isnull().sum())

Пропуски после обработки:
0


In [64]:
# Посчитать количество дубликатов ДО удаления
num_duplicates = df.duplicated(subset=['CustomerId']).sum()
print(f"Найдено дубликатов: {num_duplicates}")

# Удалить дубликаты
df = df.drop_duplicates(subset=['CustomerId'])

# Убедиться, что их больше нет
print("Дубликатов после удаления:", df.duplicated().sum())

Найдено дубликатов: 0
Дубликатов после удаления: 0


In [65]:
#Рапределяем данные по целевой переменной
target_counts = df['Exited'].value_counts()
target_ratios = df['Exited'].value_counts(normalize=True)

print("Абсолютные частоты:")
print(target_counts)
print("\nОтносительные частоты:")
print(target_ratios)

Абсолютные частоты:
Exited
0    7963
1    2037
Name: count, dtype: Int64

Относительные частоты:
Exited
0    0.7963
1    0.2037
Name: proportion, dtype: Float64


In [66]:
# Вычисляем коэффициент дисбаланса
# Предполагаем, что классы — это 0 и 1
minority_class_count = target_counts.min()
majority_class_count = target_counts.max()

# Вариант 1: отношение меньшинства к большинству (всегда <= 1)
imbalance_ratio_minority_to_majority = minority_class_count / majority_class_count

# Вариант 2: отношение большинства к меньшинству (всегда >= 1)
imbalance_ratio_majority_to_minority = majority_class_count / minority_class_count


print(f"\nКоэффициент дисбаланса (меньшинство / большинство): {imbalance_ratio_minority_to_majority:.4f}")
print(f"Коэффициент дисбаланса (большинство / меньшинство): {imbalance_ratio_majority_to_minority:.4f}")


Коэффициент дисбаланса (меньшинство / большинство): 0.2558
Коэффициент дисбаланса (большинство / меньшинство): 3.9092


целевая переменная Exited является несбалансированной


In [67]:
# Импортируем функцию для разделения данных на обучающую и тестовую выборки
from sklearn.model_selection import train_test_split

y = df['Exited'].copy()

x = df.drop(columns=['Exited']).copy()

# Разделяем данные для обучения и теста

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

In [68]:
# Импортируем класс для нормализации данных
from sklearn.preprocessing import StandardScaler

# Создаём объект StandardScaler
scaler = StandardScaler()

# Обучаем скалер на обучающих данных И преобразуем их
x_train_scaled = scaler.fit_transform(x_train)

# Преобразуем тестовые данные только с помощью уже обученного скалера
x_test_scaled = scaler.transform(x_test)


In [69]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

# Обучаем логистическую регрессию
lr_model = LogisticRegression(random_state=42, max_iter=1000)
lr_model.fit(x_train_scaled, y_train)


# Обучаем дерево решений
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(x_train_scaled, y_train)


DecisionTreeClassifier(random_state=42)

In [70]:
# Делаем предсказания
y_pred_lr = lr_model.predict(x_test_scaled)
y_pred_dt = dt_model.predict(x_test_scaled)

y_proba_lr = lr_model.predict_proba(x_test_scaled)[:, 1]
y_proba_dt = dt_model.predict_proba(x_test_scaled)[:, 1]

In [71]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report
import seaborn as sns
# import matplotlib.pyplot as plt

# Функция для вывода метрик
def print_metrics(y_true, y_pred, y_proba, model_name):
    print(f"\n=== {model_name} ===")
    print(f"Accuracy:  {accuracy_score(y_true, y_pred):.4f}")
    print(f"Precision: {precision_score(y_true, y_pred):.4f}")
    print(f"Recall:    {recall_score(y_true, y_pred):.4f}")
    print(f"F1-score:  {f1_score(y_true, y_pred):.4f}")
    print(f"ROC-AUC:   {roc_auc_score(y_true, y_proba):.4f}")

In [72]:
# 12. Вывод результатов
print_metrics(y_test, y_pred_lr, y_proba_lr, "Logistic Regression")
print_metrics(y_test, y_pred_dt, y_proba_dt, "Decision Tree")


=== Logistic Regression ===
Accuracy:  0.8060
Precision: 0.5809
Recall:    0.2254
F1-score:  0.3248
ROC-AUC:   0.7717

=== Decision Tree ===
Accuracy:  0.7950
Precision: 0.5045
Recall:    0.5411
F1-score:  0.5221
ROC-AUC:   0.7012


F1-score:  0.3248 - не соответствует требованиям
F1-score:  0.5221 - не соответствует требованиям

In [73]:
# Логистическая регрессия с балансировкой через веса
lr_balanced = LogisticRegression(class_weight='balanced', random_state=42)
dt_balanced = DecisionTreeClassifier(class_weight='balanced', random_state=42)

lr_balanced.fit(x_train_scaled, y_train)
dt_balanced.fit(x_train_scaled, y_train)

y_pred_lr_bal = lr_balanced.predict(x_test_scaled)
y_pred_dt_bal = dt_balanced.predict(x_test_scaled)
y_proba_lr_bal = lr_balanced.predict_proba(x_test_scaled)[:, 1]
y_proba_dt_bal = dt_balanced.predict_proba(x_test_scaled)[:, 1]

# Вывод метрик
print_metrics(y_test, y_pred_lr_bal, y_proba_lr_bal, "Logistic Regression (Balanced Weights)")
print_metrics(y_test, y_pred_dt_bal, y_proba_dt_bal, "Decision Tree (Balanced Weights)")


=== Logistic Regression (Balanced Weights) ===
Accuracy:  0.7067
Precision: 0.3883
Recall:    0.7246
F1-score:  0.5056
ROC-AUC:   0.7745

=== Decision Tree (Balanced Weights) ===
Accuracy:  0.7937
Precision: 0.5016
Recall:    0.5121
F1-score:  0.5068
ROC-AUC:   0.6896


После балансировки весов моделей лучше стала только Logistic Regression, а Decision Tree стало даже хуже, оба из них не достинги нормальных коэффициентов по заданию

In [74]:
from imblearn.over_sampling import SMOTE

# Применяем SMOTE только к обучающим данным
smote = SMOTE(random_state=42)
x_train_smote, y_train_smote = smote.fit_resample(x_train_scaled, y_train)

# Обучаем модели на сбалансированных данных
lr_smote = LogisticRegression(random_state=42)
dt_smote = DecisionTreeClassifier(random_state=42)

lr_smote.fit(x_train_smote, y_train_smote)
dt_smote.fit(x_train_smote, y_train_smote)

y_pred_lr_smt = lr_smote.predict(x_test_scaled)
y_pred_dt_smt = dt_smote.predict(x_test_scaled)
y_proba_lr_smt = lr_smote.predict_proba(x_test_scaled)[:, 1]
y_proba_dt_smt = dt_smote.predict_proba(x_test_scaled)[:, 1]

# Вывод метрик
print_metrics(y_test, y_pred_lr_smt, y_proba_lr_smt, "Logistic Regression (Balanced Data)")
print_metrics(y_test, y_pred_dt_smt, y_proba_dt_smt, "Decision Tree (Balanced Data)")


=== Logistic Regression (Balanced Data) ===
Accuracy:  0.7070
Precision: 0.3868
Recall:    0.7101
F1-score:  0.5009
ROC-AUC:   0.7730

=== Decision Tree (Balanced Data) ===
Accuracy:  0.7613
Precision: 0.4398
Recall:    0.5588
F1-score:  0.4922
ROC-AUC:   0.6865


После балансировки данных Logistic Regression стал выше, а Decision Tree стал хуже, чем без вмешательства, оба из них не достинги нормальных коэффициентов по заданию

In [75]:
from sklearn.ensemble import RandomForestClassifier

# Случайный лес с балансировкой классов
rf_balanced = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    n_jobs=-1
)

# Обучаем на обучающих данных
rf_balanced.fit(x_train_scaled, y_train)

# Предсказания на тестовой выборке
y_pred_rf_bal = rf_balanced.predict(x_test_scaled)
y_proba_rf_bal = rf_balanced.predict_proba(x_test_scaled)[:, 1]

# Вывод метрик
print_metrics(y_test, y_pred_rf_bal, y_proba_rf_bal, "Random Forest")


=== Random Forest ===
Accuracy:  0.8633
Precision: 0.7530
Recall:    0.5056
F1-score:  0.6050
ROC-AUC:   0.8669


После обучения модели RandomForestClassifier был достигнут нужный результат в F1-score больше чем 0.59

Accuracy - 0.8633 -Модель верно классифицирует 86% всех клиентов. Но при дисбалансе классов (например, только 20% клиентов уходят) эта метрика может вводить в заблуждение.
Precision - 0.7530 - Из всех клиентов, которых модель пометила как «уходящих» 75% действительно уйдут. Это важно, если каждое вмешательство (например, звонок менеджера, предложение бонуса) дорого.
Recall - 0.5056 - Модель находит половину всех реальных уходящих клиентов.Остальные 49% уходят незамеченными.
F1-score - 0.6050 - Гармоническое среднее Precision и Recall.
ROC AUC - 0.8669 - Высокая способность модели
разделять классы - у неё хорошая дискриминационная сила независимо от порога

ROC AUC оценивает качество модели по всем возможным порогам классификации. Он показывает, насколько хорошо модель ранжирует клиентов: уходящие — выше, остающиеся — ниже.

F1-score зависит от конкретного порога (по умолчанию — 0.5). Он фокусируется на балансе между точностью и полнотой именно при этом пороге.

Модель хорошо ранжирует, но при стандартном пороге 0.5 она осторожна: помечает как «уходящих» только тех, у кого вероятность высока → высокая Precision, но низкий Recall.
Это типично для задач с дисбалансом: модель «боится» ложно положительных срабатываний.

Рекомендация для бизнеса - настроить порог классификации под бизнес-цель.
Если важно не пропустить уходящих (например, клиенты с высоким балансом) - нужно снизить порог (например, до 0.3).
При это Recall вырастет, но Precision упадёт (больше ложных тревог).
Если каждое вмешательство дорого, то нужно повысить порог (например, до 0.7).
Тогда Precision вырастет, но многие уходящие останутся незамеченными.
Надо построить кривую Precision-Recall и выберать порог, максимизирующий F1 или бизнес-метрику (например, прибыль от удержания).